In [140]:
import numpy  as  np 
import torch 
import torch.nn as nn
import torch.nn.functional as F

##### REMEBER THIS  FROMULA 
##### OUTPUT SHAPE = [(W-K+2P)/S] + 1
#####  W = Input shape
#####  K = Kernel size
#####  p = Padding size
#####  S = Stride size


In [141]:
def fprehook(self, inputs):
    print(f"layer name: {self}")
    print(f"input shape: {inputs[0].shape}")

In [142]:
class SimpleCNN(nn.Module):
    def __init__(self,input_shape = (3,28,28)):
        super(SimpleCNN ,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3 ,out_channels=12,kernel_size=3 , stride=2 ,padding=2)
        self.conv2 = nn.Conv2d(in_channels=12 ,out_channels=32,kernel_size=3 , stride=3 ,padding=2)
        self.conv3 = nn.Conv2d(in_channels=32 ,out_channels=64,kernel_size=3 , stride=2 ,padding=2)
        self.conv4 = nn.Conv2d(in_channels=64 ,out_channels=128,kernel_size=3 , stride=3 ,padding=2)

        # Flattening the  conv layer 
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self._get_flatten_layer(input_shape),128)
        self.fc2 = nn.Linear(128,10)


    # function that  automatically  calculates the  output shape
    def _get_flatten_layer(self,shape):
        dummy_input = torch.rand(1,*shape) # here 1 mentions batch dimension 
        x = self.conv1(dummy_input)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        n_size = self.flatten(x).shape[1]
        return n_size
    

    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x,dim=1)
        return output # output spit from feed forward  layer
    



In [143]:
model = SimpleCNN(input_shape=(3,224,224))

In [144]:
model.conv1.register_forward_pre_hook(fprehook)
model.conv2.register_forward_pre_hook(fprehook)
model.conv3.register_forward_pre_hook(fprehook)
model.conv4.register_forward_pre_hook(fprehook)

In [145]:
dummy_inp = torch.rand(1,3, 224, 224)
output = model(dummy_inp)

layer name: Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
input shape: torch.Size([1, 3, 224, 224])
layer name: Conv2d(12, 32, kernel_size=(3, 3), stride=(3, 3), padding=(2, 2))
input shape: torch.Size([1, 12, 113, 113])
layer name: Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
input shape: torch.Size([1, 32, 39, 39])
layer name: Conv2d(64, 128, kernel_size=(3, 3), stride=(3, 3), padding=(2, 2))
input shape: torch.Size([1, 64, 21, 21])


In [156]:
print(output.shape)
print(f"OUTPUT FROM SOFTMAX LAYER",output)

torch.Size([1, 10])
VALUES FROM OUTPUT LAYER (0, tensor([-2.2786, -2.3007, -2.3385, -2.2799, -2.2668, -2.3515, -2.3251, -2.2764,
        -2.3787, -2.2382], grad_fn=<UnbindBackward0>)) 

